In [4]:
# Create Enviornment 
"""
$ conda install -c conda-forge astromatic-source-extractor astromatic-swarp  # install SExtractor and SWarp (optional)
$ conda create -y --name envsfft python=3.6.6  # create Python Env, other Python version is also good.
$ conda activate envsfft
$ (envsfft): pip install sfft==1.4.1  # install latest sfft via PyPI

"""

'\n$ conda install -c conda-forge astromatic-source-extractor astromatic-swarp  # install SExtractor and SWarp (optional)\n$ conda create -y --name envsfft python=3.6.6  # create Python Env\n$ conda activate envsfft\n$ (envsfft): pip install sfft==1.3.4  # install latest sfft via PyPI\n\n'

## STEP 1. specify input and output files

### A. Prepare Reference and Science images
**The image pair should be well aligned to each other.**

**Sky subtraction is REQUIRED in sparse field case.**

### B. Specify the output path of difference image

In [ ]:
import os
import numpy as np
import os.path as pa
from astropy.io import fits
from sfft.EasySparsePacket import Easy_SparsePacket

CDIR = os.path.abspath("") # get current directory
FITS_REF = CDIR + '/input_data/c4d_180806_052738_ooi_i_v1.S20.skysub.resampled.fits'  # reference 
FITS_SCI = CDIR + '/input_data/c4d_180802_062754_ooi_i_v1.S20.skysub.fits'            # science
FITS_DIFF = CDIR + '/output_data/%s.sfftdiff.fits' %(pa.basename(FITS_SCI)[:-5])    # difference

: 

## STEP 2. setting meta-parameters

### A. BACKEND_4SUBTRACT
SFFT has two backends (CPU & GPU), setting BACKEND_4SUBTRACT = 'Numpy' ('Cupy') if you want to use CPU (GPU)

### B. CUDA_DEVICE_4SUBTRACT 
If you are using GPU (i.e., 'Cupy') backend, SFFT allow you to specify which GPU device to use by specify gpu index via CUDA_DEVICE_4SUBTRACT

### C. NUM_CPU_THREADS_4SUBTRACT
If you are using CPU (i.e., 'Numpy') backend, multiple threading is allowed for speedup and recommended number of threads is 4/8. 

### D. GAIN_KEY and SATUR_KEY 

**Gain** and **Saturation** are required in FITS header of reference and science images.

**Specifying saturation level is important for crowded case**: SFFT can temporaily mask pixels contaminated by saturations using SExtractor to eliminate their impact on the subtraction performance. Saturation Level is not necessarily very accurate, but a conservative value is recommended to make sure that all possible contaminated pixels will be masked: it is ok to set an underestimated value.

**Gain**: Gain value is important for automatically identifying variables in sfft. Identified variables will be temporiarly masked to avoid their undesired impact on sfft fitting (misleading the flux).

In [2]:
# * computing backend and resourse 
BACKEND_4SUBTRACT = 'Numpy'     # FIXME {'Cupy', 'Numpy'}, Use 'Numpy' if you only have CPUs
CUDA_DEVICE_4SUBTRACT = '0'     # FIXME ONLY work for backend Cupy
NUM_CPU_THREADS_4SUBTRACT = 8   # FIXME ONLY work for backend Numpy

# * required info in FITS header
GAIN_KEY = 'GAIN'               # NOTE Keyword of Gain in FITS header
SATUR_KEY = 'SATURATE'          # NOTE Keyword of Saturation in FITS header

## STEP3. configurations for subtraction

### A. ForceConv
**ForceConv determines the direction of convolution, can be ['AUTO', 'REF', 'SCI']**

**'AUTO'**: convolve the image with smaller FWHM to avoid deconvolution.

**'REF'**: convolve the reference image, DIFF = SCI - Convolved REF. **(DIFF has consistent PSF and flux zero-point  with SCI)**.

**'SCI'**: convolve the science image, DIFF = Convolved SCI - REF. **(DIFF has consistent PSF and flux zero-point  with REF)**.

**Warning**: the estimation of image FWHM depends on point sources. SFFT identify point sources based on SExtractor photometry by perform Hough line feature detection on a plane of MAG_AUTO vs FLUX_RADIUS. FWHM estimations are commonly reliable in sparse fields where point sources are usually abundant.

### B. GKerHW 

Given half-width of matching kernel. E.g., GKerHW = 5, the matching kernel has a size 11 x 11.

**A rule of thumb: optimial GKerHW ~ 2 * max([FWHM_SCI, FWHM_REF])**

### C. KerHWRatio (default, 2)

Automatic half-width of matching kernel determined by FWHM. 
E.g., KerHWRatio = 2 with FWHM_REF = 3.0 and FWHM_SCI = 2.5, the matching kernel half-width will be 6 and size is 13 x 13. 

**Note**: KerHWRatio will be overrided when GKerHW is not None.

### D. KerPolyOrder (default, 2)
Polynomial Order of Spatial Variation of Matching Kernel. It determines the flexibility of matching kernel across the image field. 

**KerPolyOrder = 2 is commonly a good choice in most cases.**

### E. BGPolyOrder (default, 0)
Polynomial Order of Spatial Variation of Differential Background. The parameter is trivial for sparse fields as we have suctracted sky background, so we commnly set BGPolyOrder=0.

### F. ConstPhotRatio (default, True)
Image subtraction needs to align the different photometric scaling of science and reference image. One have two choices for the scaling by convolution in sfft subtraction.

**Constant scaling across the image field**: setting ConstPhotRatio = True, the sfft convolution scale the flux over the image with a constant, i.e., the sum of matching kernel does not change across the field.

**Varying scaling across the image field**: setting ConstPhotRatio = False, the sfft convolution scale the flux with spatial variation following the same form determined by the parameter KerPolyOrder. e.g., says KerPolyOrder = 2, the sum of matching kernel follows a two-ordered polynomial surface across the field.


In [3]:
# * how to subtract
ForceConv = 'AUTO'              # FIXME {'AUTO', 'REF', 'SCI'}
GKerHW = None                   # FIXME given matching kernel half width
KerHWRatio = 2.0                # FIXME Ratio of kernel half width to FWHM (typically, 1.5-2.5).
KerPolyOrder = 2                # FIXME {0, 1, 2, 3}, Polynomial degree of kernel spatial variation
BGPolyOrder = 0                 # FIXME {0, 1, 2, 3}, Polynomial degree of differential background spatial variation.
ConstPhotRatio = True           # FIXME Constant photometric ratio between images?


## STEP3+. additional configurations for subtraction 

#### G. COARSE_VAR_REJECTION (default, True)
Boolean, activate Coarse Variable Rejection (CVREJ) or not. CVREJ is used to reject variables by outlier clipping on the difference of instrumental magnitude measured on reference and science.

#### H. CVREJ_MAGD_THRESH (default, 0.12)
Magnitude threshold for CVREJ

#### I. ELABO_VAR_REJECTION (default, True)
Boolean, activate Elaborate Variable Rejection (EVREJ) or not. CVREJ is used to reject variables more carefully by taking photometric uncertainty (SExtractor FLUXERR_AUTO) into account. 

In [4]:
COARSE_VAR_REJECTION = True     # FIXME Coarse Variable Rejection? {True, False}
CVREJ_MAGD_THRESH = 0.12        # FIXME magnitude threshold for Coarse Variable Rejection
ELABO_VAR_REJECTION = True      # FIXME Elaborate Variable Rejection? {True, False}

## STEP4. run the subtraction

In [5]:
# NOTE: see complete descriptions of the parameters via help(sfft.Easy_SparsePacket)
PixA_DIFF, SFFTPrepDict = Easy_SparsePacket.ESP(FITS_REF=FITS_REF, FITS_SCI=FITS_SCI, \
    FITS_DIFF=FITS_DIFF, FITS_Solution=None, ForceConv=ForceConv, GKerHW=None, \
    KerHWRatio=KerHWRatio, KerHWLimit=(2, 20), KerPolyOrder=KerPolyOrder, \
    BGPolyOrder=BGPolyOrder, ConstPhotRatio=ConstPhotRatio, MaskSatContam=False, \
    GAIN_KEY=GAIN_KEY, SATUR_KEY=SATUR_KEY, BACK_TYPE='MANUAL', BACK_VALUE=0.0, \
    BACK_SIZE=64, BACK_FILTERSIZE=3, DETECT_THRESH=2.0, DETECT_MINAREA=5, \
    DETECT_MAXAREA=0, DEBLEND_MINCONT=0.005, BACKPHOTO_TYPE='LOCAL', \
    ONLY_FLAGS=[0], BoundarySIZE=30, XY_PriorSelect=None, Hough_MINFR=0.1, \
    Hough_PeakClip=0.7, BeltHW=0.2, PointSource_MINELLIP=0.3, MatchTol=None, \
    MatchTolFactor=3.0, COARSE_VAR_REJECTION=COARSE_VAR_REJECTION, \
    CVREJ_MAGD_THRESH=CVREJ_MAGD_THRESH, ELABO_VAR_REJECTION=ELABO_VAR_REJECTION, \
    EVREJ_RATIO_THREH=5.0, EVREJ_SAFE_MAGDEV=0.04, StarExt_iter=4, XY_PriorBan=None, \
    PostAnomalyCheck=False, PAC_RATIO_THRESH=5.0, BACKEND_4SUBTRACT=BACKEND_4SUBTRACT, \
    CUDA_DEVICE_4SUBTRACT=CUDA_DEVICE_4SUBTRACT, \
    NUM_CPU_THREADS_4SUBTRACT=NUM_CPU_THREADS_4SUBTRACT)[:2]


/Users/thomas/miniconda3/envs/envsfft/lib/python3.6/site-packages/sfft/EasySparsePacket.py:281: UserWarning: 
MeLOn REMINDER: Input images for sparse-flavor sfft should be SKY-SUBTRACTED!
  warnings.warn('\nMeLOn REMINDER: Input images for sparse-flavor sfft should be SKY-SUBTRACTED!')


MeLOn CheckPoint: TRIGGER Sparse-Flavor Auto Preprocessing [HOUGH-AUTO] MODE!

MeLOn CheckPoint [c4d_180806_052738_ooi_i_v1.S20.skysub.resampled.fits]: Run Python Wrapper of SExtractor!
MeLOn CheckPoint [c4d_180806_052738_ooi_i_v1.S20.skysub.resampled.fits]: SExtractor uses GAIN = [3.9143375010529] from keyword [GAIN]!
MeLOn CheckPoint [c4d_180806_052738_ooi_i_v1.S20.skysub.resampled.fits]: SExtractor uses SATURATION = [50335.6275904102] from keyword [SATURATE]!
MeLOn CheckPoint [c4d_180806_052738_ooi_i_v1.S20.skysub.resampled.fits]: SExtractor found [4209] sources!
MeLOn CheckPoint [c4d_180806_052738_ooi_i_v1.S20.skysub.resampled.fits]: PYSEx excludes [897 / 4209] sources by FLAGS restriction!
MeLOn CheckPoint [c4d_180806_052738_ooi_i_v1.S20.skysub.resampled.fits]: PYSEx excludes [60 / 3312] sources by boundary rejection!
MeLOn CheckPoint [c4d_180806_052738_ooi_i_v1.S20.skysub.resampled.fits]: PYSEx output catalog contains [3252] sources!
MeLOn CheckPoint: current scikit-image [0.17.2

/Users/thomas/miniconda3/envs/envsfft/lib/python3.6/site-packages/sfft/AutoSparsePrep.py:256: UserWarning: 
MeLOn WARNING: Elaborate Variable Rejection requires CORRECT GAIN in FITS HEADER!
  warnings.warn('\nMeLOn WARNING: Elaborate Variable Rejection requires CORRECT GAIN in FITS HEADER!')


MeLOn CheckPoint: SKIP, NO Prior-Banned Coordinates Given!
MeLOn CheckPoint: Active-Mask (Non-Prior-Banned SubSources) Pixel Proportion [4.32%]
MeLOn CheckPoint: TRIGGER Function Compilations of SFFT-SUBTRACTION!

 --//--//--//--//-- TRIGGER SFFT COMPILATION --//--//--//--//-- 

 ---//--- KerPolyOrder 2 | BGPolyOrder 0 | KerHW [9] ---//--- 

 --//--//--//--//-- EXIT SFFT COMPILATION --//--//--//--//-- 

MeLOn Report: Function Compilations of SFFT-SUBTRACTION TAKES [7.617 s]!
MeLOn CheckPoint: TRIGGER SFFT-SUBTRACTION!

                                __    __    __    __
                               /  \  /  \  /  \  /  \
                              /    \/    \/    \/    \
            █████████████████/  /██/  /██/  /██/  /█████████████████████████
                            /  / \   / \   / \   / \  \____
                           /  /   \_/   \_/   \_/   \    o \__,
                          / _/                       \_____/  `
                          |/
        
          

### POST CHECK 0. Basic Information

In [6]:
# 1. which side is convolved in image subtraction?
#    look into the header of difference image

CONVD = fits.getheader(FITS_DIFF, ext=0)['CONVD']
print('MeLOn CheckPoint: [%s] is convolved in sfft subtraction' %CONVD)
if CONVD == 'SCI': print('* DIFF = Convolve(SCI) - REF has PSF and zero-point aligned with [REF]!')
if CONVD == 'REF': print('* DIFF = SCI - Convolve(REF) has PSF and zero-point aligned with [SCI]!')
    
# 2. FWHM Estimations
FWHM_REF = SFFTPrepDict['FWHM_REF']
FWHM_SCI = SFFTPrepDict['FWHM_SCI']
print('\nMeLOn CheckPoint: FWHM_REF = [%.2f pix]' %FWHM_REF)
print('\nMeLOn CheckPoint: FWHM_SCI = [%.2f pix]' %FWHM_SCI)

# 3. How many sources used to fit in SFFT subtraction
AstSS = SFFTPrepDict['SExCatalog-SubSource']
print('\nMeLOn CheckPoint: [%d] Sources are selected for fitting solution in SFFT!' %(len(AstSS)))

MeLOn CheckPoint: [REF] is convolved in sfft subtraction
* DIFF = SCI - Convolve(REF) has PSF and zero-point aligned with [SCI]!

MeLOn CheckPoint: FWHM_REF = [3.10 pix]

MeLOn CheckPoint: FWHM_SCI = [4.62 pix]

MeLOn CheckPoint: [1263] Sources are selected for fitting solution in SFFT!


### POST CHECK 1. Which Pixels were Fitted by SFFT?

**For the check images, we fill random noise on the pixels which are not fitted by SFFT. That is, only the remaining pixels have contributed the solution of the matching kernel in SFFT.**

In [ ]:
from sfft.utils.SkyLevelEstimator import SkyLevel_Estimator
# open ./output_data/*.fittedPix.fits to check the pixels used in fitting
# * pixels not used have been replaced by random noise.

# for Science
PixA_SCI = SFFTPrepDict['PixA_SCI']
sky, skysig = SkyLevel_Estimator.SLE(PixA_obj=PixA_SCI)
PixA_NOSCI = np.random.normal(sky, skysig, PixA_SCI.shape)

FITS_FittedSCI = CDIR + '/output_data/%s.fittedPix.fits' %(pa.basename(FITS_SCI)[:-5])
with fits.open(FITS_SCI) as hdl:
    PixA_SCI = hdl[0].data.T
    NonActive = ~SFFTPrepDict['Active-Mask']
    PixA_SCI[NonActive] = PixA_NOSCI[NonActive]
    hdl[0].data[:, :] = PixA_SCI.T
    hdl.writeto(FITS_FittedSCI, overwrite=True)

# for Reference
PixA_REF = SFFTPrepDict['PixA_REF']
sky, skysig = SkyLevel_Estimator.SLE(PixA_obj=PixA_REF)
PixA_NOREF = np.random.normal(sky, skysig, PixA_REF.shape)

FITS_FittedREF = CDIR + '/output_data/%s.fittedPix.fits' %(pa.basename(FITS_REF)[:-5])
with fits.open(FITS_REF) as hdl:
    PixA_REF = hdl[0].data.T
    NonActive = ~SFFTPrepDict['Active-Mask']
    PixA_REF[NonActive] = PixA_NOREF[NonActive]
    hdl[0].data[:, :] = PixA_REF.T
    hdl.writeto(FITS_FittedREF, overwrite=True)

# for Difference
sky, skysig = SkyLevel_Estimator.SLE(PixA_obj=PixA_DIFF)
PixA_NODIFF = np.random.normal(sky, skysig, PixA_DIFF.shape)

FITS_FittedDIFF = CDIR + '/output_data/%s.fittedPix.fits' %(pa.basename(FITS_DIFF)[:-5])
with fits.open(FITS_DIFF) as hdl:
    PixA_DIFF = hdl[0].data.T
    NonActive = ~SFFTPrepDict['Active-Mask']
    PixA_DIFF[NonActive] = PixA_NODIFF[NonActive]
    hdl[0].data[:, :] = PixA_DIFF.T
    hdl.writeto(FITS_FittedDIFF, overwrite=True)


### POST CHECK2. check if any prominent variables survive?

In [8]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')

# open ./output_data/varcheck.pdf to see the figure
# one may deactivate any variabvle rejection and generate 
# this figure again to see the effect of our rejection.
# (that is, setting COARSE_VAR_REJECTION = False and ELABO_VAR_REJECTION = False)

AstSS = SFFTPrepDict['SExCatalog-SubSource']

plt.figure(figsize=(8, 4))
ax = plt.subplot(111)
xdata = AstSS['MAG_AUTO_REF']
exdata = AstSS['MAGERR_AUTO_REF']
ydata = AstSS['MAG_AUTO_SCI'] - AstSS['MAG_AUTO_REF']
eydata = AstSS['MAGERR_AUTO_SCI']

ax.errorbar(xdata, ydata, xerr=exdata, yerr=eydata, 
    fmt='o', markersize=2.5, color='black', mfc='#EE3277',
    capsize=2.5, elinewidth=0.5, markeredgewidth=0.1)

m = np.median(ydata) 
ml, mu = m - CVREJ_MAGD_THRESH, m + CVREJ_MAGD_THRESH
ax.hlines(y=[m, ml, mu], xmin=xdata.min(), xmax=xdata.max(), 
    linestyle='--', zorder=3, color='#1D90FF')
ax.set_xlabel(r'MAG_AUTO (REF)')
ax.set_ylabel(r'MAG_AUTO (SCI) - MAG_AUTO (REF)')
plt.savefig(CDIR + '/output_data/varcheck.pdf', dpi=300)
plt.close()
